# Check if the fcs files are in the data folder

List the files in the data folder, you should see the fcs files you want to analyze.

In [ ]:
%ls data

# Check the config file

In [ ]:
%ls *.dat

In [ ]:
!more dataCTV.dat

# Run initialization

Prepare folders for outputs and configurations

In [ ]:
import os
import fcGMM as gmm

dirEx = os.getcwd() +'/'
print('cwd', dirEx)

dirRes = dirEx+'result-data/'
if not os.path.exists(dirRes):
    os.makedirs(dirRes)
dirInit = dirEx+'init/'
if not os.path.exists(dirInit):
    os.makedirs(dirInit)
dirClean = dirEx+'cleaned/'
if not os.path.exists(dirClean):
    os.makedirs(dirClean)
dirPlots = dirEx+'plots/'
if not os.path.exists(dirPlots):
    os.makedirs(dirPlots)
    
datafile = {}
sufx = 'dataCTV'

xlab = 'V450-A'
ylab = None
zlab = None

datadir = ''
with open(sufx+'.dat') as f:
    print('read .dat')
    for line in f:
        line = line[:-1]
        #print(line)
        #print(line.split(':'))
        if os.path.isdir(dirEx+line):
            datadir = dirEx+line
        elif os.path.isfile(datadir+line.split(':')[-1]):
            try:
                datafile[int(line.split(':')[0])] = datadir+line.split(':')[-1]
            except:
                datafile[line.split(':')[0]] = datadir+line.split(':')[-1]
        else:
            print('line error', datadir+line.split(':')[-1])
            print('file does not exist',os.path.isfile(datadir+line.split(':')[-1]))
            print('dir does not exist',os.path.isdir(dirEx+line) )

names = gmm.getAq(datafile)
print('-------------------run preprocessing-------------------------')
dataframe = gmm.doPreproc(datafile, sufx, names, dirClean,xlab,ylab,zlab)

In [ ]:
!python runFcGMM.py --preprocessing --dim 1 -i CTV